# Comparing Speed of Python To Julia
(See `SpeedPython.ipynb` for comparison)

In [4]:
using BenchmarkTools

### Julia with dispatch

In [1]:
# Dispatched Function
f(x::Int64) = x^2 % 4
f(x::Float64) = f(ceil(Int64, x))
f(x::String) = f(parse(Float64, x))

# Function to apply f
function applytodata(array)
    for a in array
        f(a)
    end
end;

In [5]:
# Make a giant array of mixed types
array_len = 1_000_000
array = [s for i=1:array_len for s in ["5.5",6,5.5]]

# Run once so function will compile
applytodata(array);

# Time applying f to each element
println("Dispatch Speed")
@btime applytodata($array);

Dispatch Speed
  37.374 ms (0 allocations: 0 bytes)


### Julia that Looks Like Python

In [7]:
# Python like function
function f_py(x)
    if isa(x,String)
        x = parse(Float64,x)
    end
    if isa(x,Float64)
        x = ceil(Int64,x)
    end
    x^2 % 4
end

function applytodata_py(array)
    for a in array
        f_py(a)
    end
end;


In [8]:
# Make a giant array of mixed types
array = [s for i=1:1000000 for s in ["5.5",6,5.5]]

# Run once so function will compile
applytodata_py(array);

# Time applying f_py to each element
println("Python like speed")
@btime applytodata_py($array);

Python like speed
  80.459 ms (0 allocations: 0 bytes)


In [10]:
Test = ["1.5", 2.5, 3]

Repeat = [10^6:10^7:10^8+1;]

10-element Vector{Int64}:
  1000000
 11000000
 21000000
 31000000
 41000000
 51000000
 61000000
 71000000
 81000000
 91000000

In [20]:
ans_time = []
ans_allo = []
for R in Repeat
    array = [s for i=1:R for s in ["1.5", 2.5, 3]]
    time = @timed applytodata(array)
    push!(ans_time, time[2])
    push!(ans_allo, time[3])
    println("Array length: $R Seconds: $(time[2])")
end

Array length: 1000000 Seconds: 0.039610665
Array length: 11000000 Seconds: 0.427003753


LoadError: InterruptException: